In [8]:
import pandas as pd
import numpy as np

In [20]:
diabetes1 = pd.read_csv('C:\IDE\python_homework\Пайтон для анализа данных\Модуль 7. Очистка данных\data\diabetes_data.csv')
diabetes1.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


In [21]:
dupl_columns = list(diabetes1.columns)
mask = diabetes1.duplicated(subset=dupl_columns)
diabetes1_duplicates = diabetes[mask]

diabetes1 = diabetes1.drop_duplicates(subset=dupl_columns)

diabetes1.shape

C:\Users\troyd\AppData\Local\Temp\ipykernel_5864\2386351587.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  diabetes1_duplicates = diabetes[mask]


(768, 10)

In [24]:
low_information_cols = [] 

#цикл по всем столбцам
for col in diabetes1.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes1[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes1[col].nunique() / diabetes1[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
        
diabetes1 = diabetes1.drop('Gender', axis=1)

Gender: 100.0% одинаковых значений


In [25]:
diabetes1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 60.0 KB


In [26]:
def XXXXX(cifra):
    if cifra == 0 or cifra == 0.0:
        return np.nan
    else:
        return cifra
    
lisssst = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

for col in lisssst:
    diabetes1[col] = diabetes1[col].apply(XXXXX)

In [29]:
diabetes1
cols_null_percent = diabetes1.isnull().sum()
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null)
diabetes1['Insulin'].shape[0]

Insulin          374
SkinThickness    227
BloodPressure     35
BMI               11
Glucose            5
dtype: int64

768

In [34]:
combine_data = diabetes1.copy()

#отбрасываем столбцы с числом пропусков более 30% (100-70)
n = diabetes1.shape[0] #число строк в таблице
thresh = n*0.7
combine_data = combine_data.dropna(thresh=thresh, axis=1)
m = combine_data.shape[1] #число признаков после удаления столбцов
combine_data = combine_data.dropna(thresh=m-2, axis=0)
combine_data.shape
diabetes1 = combine_data

In [39]:
diabetes1.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98.0,58.0,33.0,34.0,0.430,43,0
1,2,112.0,75.0,32.0,35.7,0.148,21,0
2,2,108.0,64.0,NaN,30.8,0.158,21,0
3,8,107.0,80.0,NaN,24.6,0.856,34,0
4,7,136.0,90.0,NaN,29.9,0.210,50,0


In [42]:
values = {
    'Pregnancies': diabetes1['Pregnancies'].median(),
    'Glucose': diabetes1['Glucose'].median(),
    'BloodPressure': diabetes1['BloodPressure'].median(),
    'SkinThickness': diabetes1['SkinThickness'].median(),
    'BMI': diabetes1['BMI'].median(),
    'DiabetesPedigreeFunction': diabetes1['DiabetesPedigreeFunction'].median(),
    'Age': diabetes1['Age'].median(),
    'Outcome': diabetes1['Outcome'].median()
}
#заполняем оставшиеся записи константами в соответствии со словарем values
combine_data1 = diabetes1.fillna(values)
#выводим результирующую долю пропусков
combine_data1['SkinThickness'].mean()

diabetes1 = combine_data1

In [46]:
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned
#✍️ Применим эту функцию к таблице sber_data и признаку full_sq, а также выведем размерности результатов:

outliers, cleaned = outliers_iqr(diabetes1, 'DiabetesPedigreeFunction')

outliers.shape

(29, 8)

In [47]:
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned
outliers, cleaned = outliers_iqr_mod(diabetes1, 'DiabetesPedigreeFunction', log_scale=True)
print(outliers.shape[0])

0


In [44]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned
#Применим эту функцию к таблице sber_data и признаку mkad_km, а также выведем размерности результатов:

outliers, cleaned = outliers_z_score(diabetes1, 'SkinThickness', log_scale=False)
outliers

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Outcome
30,0,162.0,76.0,56.0,53.2,0.759,25,1
36,2,197.0,70.0,99.0,34.7,0.575,62,1
382,0,180.0,78.0,63.0,59.4,2.420,25,1
444,0,100.0,88.0,60.0,46.8,0.962,31,0


In [ ]:
#создаём копию исходной таблицы
combine_data = diabetes1.copy()

#отбрасываем столбцы с числом пропусков более 30% (100-70)
n = diabetes1.shape[0] #число строк в таблице
thresh = n*0.7
combine_data = combine_data.dropna(thresh=thresh, axis=1)

#отбрасываем строки с числом пропусков более 2 в строке
m = combine_data.shape[1] #число признаков после удаления столбцов
combine_data = combine_data.dropna(thresh=m-2, axis=0)

#создаём словарь 'имя_столбца': число (признак), на который надо заменить пропуски 
values = {
    'life_sq': combine_data['full_sq'],
    'metro_min_walk': combine_data['metro_min_walk'].median(),
    'metro_km_walk': combine_data['metro_km_walk'].median(),
    'railroad_station_walk_km': combine_data['railroad_station_walk_km'].median(),
    'railroad_station_walk_min': combine_data['railroad_station_walk_min'].median(),
    'preschool_quota': combine_data['preschool_quota'].mode()[0],
    'school_quota': combine_data['school_quota'].mode()[0],
    'floor': combine_data['floor'].mode()[0]
}
#заполняем оставшиеся записи константами в соответствии со словарем values
combine_data = combine_data.fillna(values)
#выводим результирующую долю пропусков
display(combine_data.isnull().mean())
# Выведем результирующее число строк и столбцов:


print(combine_data.shape)

In [19]:
drop_data = diabetes.copy()
drop_data = drop_data.dropna(subset=lisssst, how='any', axis=0)

diabetes = drop_data
